In [16]:
# Problems 5-7 on https://work.caltech.edu/homework/hw2.pdf

import random
import numpy as np

def runs(n, run_count):
    total_e_in = 0
    total_e_out = 0

    for i in range(0, run_count):
        e_in, e_out = run(n)
        total_e_in += e_in
        total_e_out += e_out
    
    avg_e_in = total_e_in / run_count
    avg_e_out = total_e_out / run_count

    return avg_e_in, avg_e_out


def runs_pla(n, run_count):
    total_iterations = 0

    for i in range(0, run_count):
        iterations = run_pla(n)
        total_iterations += iterations
    
    avg_iterations = total_iterations / run_count

    return avg_iterations


def run(n):
    line = generate_line()
    slope, intercept = line
    rng = np.random.default_rng()
    xs = rng.uniform(-1,1,(n, 2))
    line_heights = (slope * xs[:, 0]) + intercept
    ys = np.where(xs[:, 1] > line_heights, 1, -1)
    xs = np.hstack((np.ones((n,1)), xs)) # Set x_0 = 1 for all xs

    weights = pseudo_inverse(xs) @ ys

    predicted_ys = np.sign(xs @ weights)
    misclassified_points = np.flatnonzero(predicted_ys != ys)

    # E_in = the fraction of in-sample points which got classified incorrectly
    e_in = len(misclassified_points) / n
    
    # Hw instructs to use 1000 out of sample test points
    xs = rng.uniform(-1,1,(1000, 2))
    line_heights = (slope * xs[:, 0]) + intercept
    ys = np.where(xs[:, 1] > line_heights, 1, -1)
    xs = np.hstack((np.ones((1000,1)), xs)) # set x_0's = 1
    predicted_ys = np.sign(xs @ weights)
    misclassified_count = len(np.flatnonzero(predicted_ys != ys))

    # E_out = number of misclassified out-of-sample points / total number of out-of-sample points
    e_out = misclassified_count / 1000

    return e_in, e_out

def pseudo_inverse(xs):
    return np.linalg.inv(xs.T @ xs) @ xs.T

def run_pla(n):
    line = generate_line()
    slope, intercept = line
    rng = np.random.default_rng()
    xs = rng.uniform(-1,1,(n, 2))
    line_heights = (slope * xs[:, 0]) + intercept
    ys = np.where(xs[:, 1] > line_heights, 1, -1)
    xs = np.hstack((np.ones((n,1)), xs)) # Set x_0 = 1 for all xs

    weights = pseudo_inverse(xs) @ ys

    iterations = 0 # keep track of how many iterations of the PLA we have to do
    while True:
        # Predict y's with current weights, and note which ones are misclassified
        predicted_ys = np.sign(xs @ weights)
        misclassified_points = np.flatnonzero(predicted_ys != ys)

        # break if no more misclassified points
        if len(misclassified_points) == 0:
            break
        
        iterations += 1

        # Pick a random misclassified point
        misclassified_i = random.choice(misclassified_points)
        x = xs[misclassified_i]
        y = ys[misclassified_i]
        # update weights
        weights = weights + (y * x) 
    
    return iterations

# Returns the intercept and slope of randomly generated line
def generate_line():
    point_1 = generate_point()
    point_2 = generate_point()
    return slope_and_intercept(point_1,point_2)    

def generate_point():
    return [random.uniform(-1,1), random.uniform(-1,1)]

def slope_and_intercept(point_1, point_2):
    point_1, point_2 = sorted([point_1, point_2])
    slope = (point_2[1] - point_1[1]) / (point_2[0] - point_1[0])
    intercept = point_1[1] - (slope * point_1[0])
    return slope, intercept

print(runs(100, 1000))
print(runs_pla(10, 1000))


(0.03731000000000007, 0.04710700000000009)
3.915
